In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from osgeo import gdal
from osgeo import gdalconst
import os
from osgeo import ogr
from osgeo import osr
import fiona
from ops.ops import load_json
from tqdm.notebook import tqdm_notebook
from osgeo import gdal_array
from skimage.morphology import disk, dilation, erosion
import numpy as np

In [2]:
#image_folder = 'img/GEE-imgs'

paths = load_json(os.path.join('conf', 'paths.json'))
shp = load_json(os.path.join('conf', 'shp.json'))
conf = load_json(os.path.join('conf', 'conf.json'))
shp_path = paths['shp']

img_source = conf['img_source']
grid_save = os.path.join('shp', f"tiles.shp")


In [3]:
image_folder = os.path.join('img', 'opt')
labels_folder = os.path.join('img', 'labels')

image_p = os.path.join(image_folder, '2017', f'PA_S2_2017_B1_B2_B3_crop.tif')

v_def_until_2007 = ogr.Open(os.path.join(shp_path, 'accumulated_deforestation_area.shp'))
l_def_until_2007 = v_def_until_2007.GetLayer()

v_hidro = ogr.Open(os.path.join(shp_path, 'hidro_area.shp'))
l_hidro = v_hidro.GetLayer()

v_no_forest = ogr.Open(os.path.join(shp_path, 'no_forest_area.shp'))
l_no_forest = v_no_forest.GetLayer()

#v_def_2008_2017 = ogr.Open(os.path.join(shp_path, 'deforestation_until_2017.shp'))
#l_def_2008_2017 = v_def_2008_2017.GetLayer()

v_def_yearly = ogr.Open(os.path.join(shp_path, 'yearly_deforestation_area.shp'))
l_def_yearly = v_def_yearly.GetLayer()

data = gdal.Open(image_p, gdalconst.GA_ReadOnly)

geo_transform = data.GetGeoTransform()
#x_min = geo_transform[0]
#y_max = geo_transform[3]
#x_max = x_min + geo_transform[1] * data.RasterXSize
#y_min = y_max + geo_transform[5] * data.RasterYSize
x_res = data.RasterXSize
y_res = data.RasterYSize

crs = data.GetSpatialRef()
proj = data.GetProjection()

pixel_width = geo_transform[1]

## Year ref

In [5]:
ref_year = 2019

output = os.path.join(labels_folder, f'label_{ref_year}.tif')

target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
#target_ds.SetGeoTransform((x_min, geo_transform[1], 0, y_min, 0, geo_transform[5]))
target_ds.SetGeoTransform(geo_transform)
target_ds.SetSpatialRef(crs)
target_ds.SetProjection(proj)
band = target_ds.GetRasterBand(1)
#NoData_value = 0
#band.SetNoDataValue(NoData_value)
band.FlushCache()
where_past = f'"year"<={ref_year-1}'
where_ref = f'"year"={ref_year}'
gdal.RasterizeLayer(target_ds, [1], l_hidro, burn_values=[2])
gdal.RasterizeLayer(target_ds, [1], l_no_forest, burn_values=[2])
gdal.RasterizeLayer(target_ds, [1], l_def_until_2007, burn_values=[2])

l_def_yearly.SetAttributeFilter(where_past)
gdal.RasterizeLayer(target_ds, [1], l_def_yearly, burn_values=[2])

l_def_yearly.SetAttributeFilter(where_ref)
gdal.RasterizeLayer(target_ds, [1], l_def_yearly, burn_values=[1])
rasterized_data = target_ds.ReadAsArray() 
defor_data = rasterized_data == 1
defor_data = defor_data.astype(np.uint8)

border_data = dilation(defor_data, disk(1)) - erosion(defor_data, disk(1))
rasterized_data[border_data==1] = 2

target_ds.GetRasterBand(1).WriteArray(rasterized_data)
target_ds = None